# 填充和步幅
:label:`sec_padding`

回想:numref:`fig_correlation`中的卷积示例。
输入的高度和宽度均为3，
而卷积核的高度和宽度均为2，
从而产生一个$2\times2$的输出表示。
假设输入形状为$n_\textrm{h}\times n_\textrm{w}$
且卷积核形状为$k_\textrm{h}\times k_\textrm{w}$，
则输出形状将为$(n_\textrm{h}-k_\textrm{h}+1) \times (n_\textrm{w}-k_\textrm{w}+1)$：
我们只能移动卷积核直到它没有更多的像素来应用卷积。

在接下来的内容中，我们将探讨一些技术，
包括填充和带步幅的卷积，
这些技术提供了对输出大小的更多控制。
作为动机，请注意由于卷积核通常
宽度和高度都大于1，
在应用许多连续的卷积之后，
我们往往会得到比输入小得多的输出。
如果我们从一个$240 \times 240$像素的图像开始，
十个$5 \times 5$的卷积层
会将图像减少到$200 \times 200$像素，
切掉了图像的30%，
并随之抹去了原始图像边界上的任何有趣信息。
*填充*是处理此问题最常用的工具。
在其他情况下，我们可能希望大幅降低维度，
例如，如果发现原始输入分辨率难以处理。
*带步幅的卷积*是一种可以帮助解决这些问题的流行技术。

In [1]:
import torch
from torch import nn

## 填充

如上所述，应用卷积层时的一个棘手问题是，我们往往会丢失图像边缘的像素。考虑:numref:`img_conv_reuse`，它描述了卷积核大小和图像内位置对像素利用率的影响。角落的像素几乎完全没有被使用。

![不同大小的卷积（$1 \times 1$、$2 \times 2$ 和 $3 \times 3$）的像素利用率。](../img/conv-reuse.svg)
:label:`img_conv_reuse`

由于我们通常使用小的卷积核，
对于任何给定的卷积
我们可能只会丢失少数几个像素，
但随着我们应用多个连续的卷积层，
这些损失会累积起来。
解决这个问题的一个直接方法
是在输入图像的边界周围添加额外的填充像素，
从而增加图像的有效尺寸。
通常，我们将这些额外像素的值设为零。
在:numref:`img_conv_pad`中，我们对一个$3 \times 3$的输入进行填充，
将其尺寸增加到$5 \times 5$。
相应的输出则增加到一个$4 \times 4$的矩阵。
阴影部分是第一个输出元素以及用于计算该输出的输入和卷积核张量元素：$0\times0+0\times1+0\times2+0\times3=0$。

![带填充的二维互相关。](../img/conv-pad.svg)
:label:`img_conv_pad`

一般来说，如果我们总共添加$p_\textrm{h}$行填充
（大约一半在顶部，一半在底部）
和总共$p_\textrm{w}$列填充
（大约一半在左侧，一半在右侧），
那么输出形状将是

$$(n_\textrm{h}-k_\textrm{h}+p_\textrm{h}+1)\times(n_\textrm{w}-k_\textrm{w}+p_\textrm{w}+1)。$$

这意味着输出的高度和宽度将分别增加$p_\textrm{h}$和$p_\textrm{w}$。

在许多情况下，我们会希望设置$p_\textrm{h}=k_\textrm{h}-1$和$p_\textrm{w}=k_\textrm{w}-1$
以使输入和输出具有相同的高度和宽度。
这将使在网络构建时更容易预测每一层的输出形状。
假设这里$k_\textrm{h}$是奇数，
我们将在高度的两侧各填充$p_\textrm{h}/2$行。
如果$k_\textrm{h}$是偶数，一种可能是
在输入的顶部填充$\lceil p_\textrm{h}/2\rceil$行
并在底部填充$\lfloor p_\textrm{h}/2\rfloor$行。
我们将在宽度的两侧以相同的方式进行填充。

CNN通常使用高度和宽度为奇数的卷积核，
例如1、3、5或7。
选择奇数大小的卷积核的好处是
我们可以保持维度不变
同时在上下左右填充相同数量的行和列。

此外，这种使用奇数卷积核
并通过填充精确保持维度的做法
还提供了管理上的便利。
对于任意二维张量`X`，
当卷积核的大小为奇数
且所有边的填充行数和列数相同时，
产生的输出与输入具有相同的高度和宽度，
我们知道输出`Y[i, j]`是由输入和卷积核的互相关
在窗口中心位于`X[i, j]`处计算得出的。

在下面的例子中，我们创建了一个高度和宽度均为3的二维卷积层
并（**在所有边上应用1个像素的填充。**）
给定一个高度和宽度均为8的输入，
我们发现输出的高度和宽度也是8。

In [2]:
# We define a helper function to calculate convolutions. It initializes the
# convolutional layer weights and performs corresponding dimensionality
# elevations and reductions on the input and output
def comp_conv2d(conv2d, X):
    # (1, 1) indicates that batch size and the number of channels are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Strip the first two dimensions: examples and channels
    return Y.reshape(Y.shape[2:])

# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

当卷积核的高度和宽度不同时，我们可以通过[**为高度和宽度设置不同的填充数**]来使输出和输入具有相同的高度和宽度。

In [3]:
# We use a convolution kernel with height 5 and width 3. The padding on either
# side of the height and width are 2 and 1, respectively
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## 步幅

在计算互相关时，
我们从输入张量的左上角开始放置卷积窗口，
然后将其向右和向下逐位置滑动。
在前面的例子中，我们默认每次滑动一个元素。
然而，有时为了计算效率
或者因为我们希望进行下采样，
我们会让窗口一次滑过多个元素，
跳过中间的位置。当卷积核较大时，这一点特别有用，因为它可以捕获底层图像的较大区域。

我们将每次滑动经过的行数和列数称为*步幅*。
到目前为止，我们在高度和宽度上都使用了1的步幅。
有时，我们可能希望使用更大的步幅。
:numref:`img_conv_stride` 显示了一个二维互相关操作，
其垂直方向上的步幅为3，水平方向上的步幅为2。
阴影部分是输出元素以及用于输出计算的输入和核张量元素：$0\times0+0\times1+1\times2+2\times3=8$, $0\times0+6\times1+0\times2+0\times3=6$。
我们可以看到，当生成第一列的第二个元素时，
卷积窗口向下移动三行。
当生成第一行的第二个元素时，
卷积窗口向右移动两列。
当卷积窗口继续在输入上向右滑动两列时，
没有输出，因为输入元素无法填满窗口
（除非我们添加另一列填充）。

![高度和宽度分别为3和2的步幅的互相关](../img/conv-stride.svg)
:label:`img_conv_stride`

一般来说，当高度步幅为$s_\textrm{h}$
且宽度步幅为$s_\textrm{w}$时，输出形状为

$$\lfloor(n_\textrm{h}-k_\textrm{h}+p_\textrm{h}+s_\textrm{h})/s_\textrm{h}\rfloor \times \lfloor(n_\textrm{w}-k_\textrm{w}+p_\textrm{w}+s_\textrm{w})/s_\textrm{w}\rfloor.$$

如果设置$p_\textrm{h}=k_\textrm{h}-1$ 和 $p_\textrm{w}=k_\textrm{w}-1$，
则输出形状可以简化为
$\lfloor(n_\textrm{h}+s_\textrm{h}-1)/s_\textrm{h}\rfloor \times \lfloor(n_\textrm{w}+s_\textrm{w}-1)/s_\textrm{w}\rfloor$。
更进一步，如果输入的高度和宽度
能被高度和宽度的步幅整除，
则输出形状将为$(n_\textrm{h}/s_\textrm{h}) \times (n_\textrm{w}/s_\textrm{w})$。

下面，我们[**将高度和宽度的步幅都设为2**]，
从而将输入的高度和宽度减半。

In [4]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

让我们来看一个（稍微复杂一点的例子）。

In [5]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

## 摘要与讨论

填充可以增加输出的高度和宽度。这通常用于使输出的高度和宽度与输入相同，以避免输出不希望的缩小。此外，它确保所有像素被同等频繁地使用。通常我们在输入高度和宽度的两侧选择对称填充。在这种情况下，我们称之为 $(p_\textrm{h}, p_\textrm{w})$ 填充。最常见的是我们设置 $p_\textrm{h} = p_\textrm{w}$，此时我们简单地说选择填充 $p$。

类似的约定也适用于步幅。当水平步幅 $s_\textrm{h}$ 和垂直步幅 $s_\textrm{w}$ 相等时，我们简单地说步幅 $s$。步幅可以降低输出的分辨率，例如将输出的高度和宽度减少为输入的高度和宽度的 $1/n$（对于 $n > 1$）。默认情况下，填充为0，步幅为1。

到目前为止，我们讨论的所有填充都只是用零扩展图像。这具有显著的计算优势，因为很容易实现。此外，操作符可以设计成隐式利用这种填充而无需分配额外的内存。同时，它允许CNN通过学习“空白”位置来编码图像中的隐含位置信息。除了零填充之外还有很多替代方案。:citet:`Alsallakh.Kokhlikyan.Miglani.ea.2020` 提供了这些方案的全面概述（尽管没有明确说明在什么情况下使用非零填充，除非出现伪影）。

## 练习

1. 给定本节中最后一个代码示例，其中内核大小为 $(3, 5)$，填充为 $(0, 1)$，步幅为 $(3, 4)$，
   计算输出形状以检查其是否与实验结果一致。
1. 对于音频信号，步幅为2对应什么？
1. 实现镜像填充，即边界值被简单镜像以扩展张量。
1. 步幅大于1有哪些计算优势？
1. 步幅大于1可能有哪些统计优势？
1. 你如何实现 $\frac{1}{2}$ 的步幅？它对应什么？何时会很有用？

[讨论](https://discuss.d2l.ai/t/68)